<a href="https://colab.research.google.com/github/Di-anaBF/Cropland-Mapping/blob/main/KenyaCrop_Type_Process_Labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Labels

**Script Author**: Ivan Zvonkov
**Country Specific Upadate (Kenya)**: Diana B. Frimpong

**Description**: Cleans collected data from Kenya


## 1. Load reference label sets

In [32]:
import pandas as pd

In [33]:
# Load in csv files
df1 = pd.read_csv("/content/Crop type data.csv")
df2 = pd.read_csv("/content/Crop type datatwo.csv")


In [34]:
len(df1), len(df2)

(1609, 1361)

In [35]:
# TASK 1: Add code to analyze contents of a single row from data frame 1
#########################
# Your code below


#########################

In [11]:
# Plot the points
import plotly.express as px

fig = px.scatter_mapbox(df1, lat='lat',lon='lon', hover_name="plotid", zoom=6)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

## 2. Clean Reference Data

In [36]:
# Stack all two dataframes on top of one another
# so they can be simultaneously analyzed
all_dfs = pd.concat([df1, df2])
len(all_dfs)

2970

In [37]:
from datetime import date

all_dfs["start_date"] = date(2023, 1, 1)
all_dfs["end_date"] = date(2024, 1, 1)

In [ ]:
# TASK 2: Check if there are there any rows with no label? If yes how many?
#########################
# Your code below



#########################

In [47]:
# Remove rows where no label has been added
all_dfs_clean = all_dfs[~all_dfs["main_crop"].isna()].copy()

In [48]:
# Convert the label to a number for easier processing
all_dfs_clean["is_maize"] = (all_dfs_clean["main_crop"] == "maize").astype(int)

In [49]:
all_dfs_clean["is_maize"].value_counts()

is_maize
1    2288
0     682
Name: count, dtype: int64

## 3. Process Labeler Agreement

In [ ]:
# Create column for keeping track of labelers
#all_dfs_clean["num_labeler"] = 1

In [50]:
# Combine all rows which have the same latitude and longitude
df = all_dfs_clean.groupby(
    ["lon", "lat", "start_date", "end_date"],
    as_index=False,
    sort=False
).agg({"is_maize": "mean"})

In [51]:
# Analyze distribution of labels
df[["is_maize"]].value_counts()

is_maize
1.0         2288
0.0          682
Name: count, dtype: int64

In [52]:
# Remove all points with equal disagreement
df = df[df["is_maize"] != 0.5].copy().reset_index()

In [53]:
# Round the crop values e.g. 0.3333 becomes 0.0.
df["is_maize"] = (df["is_maize"] > 0.5).astype(int)

In [ ]:
# TASK 3: How many crops are there in the cleaned label set?
#########################
# Your code below



#########################

## 4. Split into training and test sets

Since this is set is larger

In [54]:
# Split into train and test sets
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=891, random_state=0)


In [55]:
len(train_df), len(test_df)

(2079, 891)

In [56]:
train_df.to_csv("train-kenyacroptype.csv", index=False)
test_df.to_csv("test-kenyacroptype.csv", index=False)